### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [3]:
reviews.shape

(812217, 5)

In [5]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Western,Thriller,Sport,...,Biography,Game-Show,History,Music,Adult,Comedy,Film-Noir,Documentary,Musical,Fantasy
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,111161,10,1373234211,2013-07-07 21:56:51
1,1,117060,7,1373415231,2013-07-10 00:13:51
2,1,120755,6,1373424360,2013-07-10 02:46:00
3,1,317919,6,1373495763,2013-07-10 22:36:03
4,1,454876,10,1373621125,2013-07-12 09:25:25


#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [17]:
movie_ratings = reviews.groupby('movie_id')['rating']
last_rating = reviews.groupby('movie_id').max()['date']

In [22]:
last_rating.head()

movie_id
8     2014-04-08 18:20:11
10    2014-10-09 18:15:53
12    2015-08-10 23:16:19
25    2017-02-27 10:04:59
91    2019-07-12 10:48:46
Name: date, dtype: object

In [20]:
avg_rating = movie_ratings.mean()
num_rating = movie_ratings.count()

In [25]:
num_rating.head()

movie_id
8     1
10    1
12    1
25    1
91    3
Name: rating, dtype: int64

In [26]:
rating_counts_df = pd.DataFrame({'avg_rating': avg_rating, 'count': num_rating, 'last_rating': last_rating})

In [27]:
rating_counts_df.head()

,avg_rating,count,last_rating
movie_id,,,
8,5.0,1,2014-04-08 18:20:11
10,10.0,1,2014-10-09 18:15:53
12,10.0,1,2015-08-10 23:16:19
25,8.0,1,2017-02-27 10:04:59
91,6.0,3,2019-07-12 10:48:46


In [28]:
movies.set_index('movie_id').head()

,movie,genre,date,1800's,1900's,2000's,Western,Thriller,Sport,Short,...,Biography,Game-Show,History,Music,Adult,Comedy,Film-Noir,Documentary,Musical,Fantasy
movie_id,,,,,,,,,,,,,,,,,,,,,
8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
def create_ranked_df(movies, reviews):
    movie_ratings = reviews.groupby('movie_id')['rating']
    avg_rating = movie_ratings.mean()
    num_rating = movie_ratings.count()
    last_rating = reviews.groupby('movie_id').max()['date']
    
    rating_counts_df = pd.DataFrame({'avg_rating': avg_rating, 'count': num_rating, 'last_rating': last_rating})
    
    movie_recs = movies.set_index('movie_id').join(rating_counts_df)
    
    ranked_df = movie_recs.sort_values(['avg_rating', 'count', 'last_rating'], ascending=False)
    ranked_df = ranked_df[ranked_df['count']>=5]
    
    return ranked_df

def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    top_movies = list(ranked_movies['movie'][:n_top])
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [7]:
# Put your solutions for each of the cases here
ranked = create_ranked_df(movies, reviews)

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20, ranked)# Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5, ranked)# Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100, ranked)# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35, ranked)# Your solution list here



In [8]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [9]:
ranked.head()

,movie,genre,date,1800's,1900's,2000's,Western,Thriller,Sport,Short,...,Music,Adult,Comedy,Film-Noir,Documentary,Musical,Fantasy,avg_rating,count,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,2015,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,10.0,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,10.0,28,2016-01-08 00:44:43
6662050,Five Minutes (2017),Short|Comedy,2017,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,10.0,22,2019-04-20 22:29:19
2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.0,10,2015-05-10 22:56:01
2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,10.0,6,2016-01-23 00:30:44


In [29]:
ranked[['Short','Comedy']].head()

,Short,Comedy
movie_id,,
4921860,0,1
5262972,1,1
6662050,1,1
2737018,0,0
2560840,0,0


In [30]:
ranked[['Short','Comedy']].sum(axis=1).head()

movie_id
4921860    1
5262972    2
6662050    2
2737018    0
2560840    0
dtype: int64

In [33]:
def popular_recommendations_filtered(user_id, n_top, ranked_movies, years=[], genres=[]):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    
    filtered = ranked_movies
    if len(years) > 0 :
        filtered = ranked_movies[ranked_movies['date'].isin(years)]
    
    if len(genres) > 0 :
        genre_filtered = filtered[genres].sum(axis=1)
        filtered = filtered.loc[genre_filtered > 0, :]
        
    # Do stuff
    top_movies = list(filtered['movie'][:n_top])
    return top_movies # a list of the n_top movies as recommended

In [37]:
filtered = popular_recommendations_filtered(1, 20, ranked, years=[2015], genres=['Short'])
filtered

['Avengers: Age of Ultron Parody (2015)',
 'Make Like a Dog (2015)',
 'World of Tomorrow (2015)',
 'Kung Fury (2015)',
 'Borrowed Time (2015)',
 'A Girl in the River: The Price of Forgiveness (2015)',
 "Sanjay's Super Team (2015)",
 'Stutterer (2015)',
 'Frozen Fever (2015)',
 'Ellis (2015)']